# **Step1_AI면접관 Agent v1.0**

## **1. 환경준비**

### (1) 구글 드라이브

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(project_genai)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

### (2) 라이브러리

In [ ]:
!pip install -r /content/drive/MyDrive/project_genai/requirements.txt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

### (3) OpenAI API Key 확인
* api_key.txt 파일에 다음의 키를 등록하세요.
    * OPENAI_API_KEY
    * NGROK_AUTHTOKEN

In [ ]:
import os

def load_api_keys(filepath="api_key.txt"):
    with open(filepath, "r") as f:
        for line in f:
            line = line.strip()
            if line and "=" in line:
                key, value = line.split("=", 1)
                os.environ[key.strip()] = value.strip()

path = '/content/drive/MyDrive/project_genai/'
# API 키 로드 및 환경변수 설정
load_api_keys(path + 'api_key.txt')

In [ ]:
print(os.environ['OPENAI_API_KEY'][:30])

sk-proj-vrfailtD8W332aZ7MRrMIP


## **2. App.py**

* 아래 코드에, Step1 혹은 고도화 된 Step2 파일 코드를 붙인다.
    * 라이브러리
    * 함수들과 그래프
* Gradio 코드는 그대로 사용하거나 일부 수정 가능

In [ ]:
%%writefile app.py

####### 여러분의 함수와 클래스를 모드 여기에 붙여 넣읍시다. #######
## 1. 라이브러리 로딩 ---------------------------------------------
import pandas as pd
import numpy as np
import os
import openai
import random
import ast
import fitz
from docx import Document as docxdocu
from langchain_core.documents import Document

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from typing import Annotated, Literal, Sequence, TypedDict, List, Dict
from langchain import hub
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.output_parsers import CommaSeparatedListOutputParser
from langgraph.graph import StateGraph, END

## ---------------- 1단계 : 사전준비 ----------------------

# 1) 파일 입력 --------------------
def extract_text_from_file(file_path: str) -> str:
    ext = os.path.splitext(file_path)[1].lower()
    if ext == ".pdf":
        doc = fitz.open(file_path)
        text = "\n".join(page.get_text() for page in doc)
        doc.close()
        return text
    elif ext == ".docx":
        doc = docxdocu(file_path)
        return "\n".join(p.text for p in doc.paragraphs if p.text.strip())
    else:
        raise ValueError("지원하지 않는 파일 형식입니다. PDF 또는 DOCX만 허용됩니다.")

# 2) State 선언 --------------------
class InterviewState(TypedDict):
    # 고정 정보
    resume_text: str
    resume_summary: str
    resume_keywords: List[str]
    question_strategy: Dict[str, Dict]

    # 인터뷰 로그
    current_question: str
    current_answer: str
    current_strategy: str
    conversation: List[Dict[str, str]]
    evaluation : List[Dict[str, str]]
    next_step : str

# 3) resume 분석 --------------------
def analyze_resume(state: InterviewState) -> InterviewState:
    llm = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.3, model_kwargs={"top_p":0.8})

    # 핵심 요약
    s_msg = "너는 자소서를 잘 요약하는 유명한 면접 컨설턴트야."
    h_msg = "{resume}을 자세히 분석하여, 면접에 중요한 부분을 위주로 핵심을 요약해줘."

    chat_promt = ChatPromptTemplate.from_messages([
            ("system",s_msg), ("human",h_msg),
    ])

    messages = chat_promt.format_messages(resume=state['resume_text'])
    response = llm(messages)
    resume_summary = response.content

    # 주요 키워드
    parser = CommaSeparatedListOutputParser()
    prompt = ChatPromptTemplate.from_messages([
        ("system", "너는 자소서에서 핵심 키워드를 잘 뽑아내는 유명한 면접 컨설턴트야."),
        ("human", """
        면접을 위한 질문 전략을 수립하기 위해, 다음의 자소서 내용에서 중요한 키워드 10개를 콤마로 구분해서 말해줘.
        자소서 내용: {resume}
        """),
        ("system", "{format_instructions}")
    ])

    formatted_messages = prompt.format_messages(
        resume=state["resume_text"],
        format_instructions=parser.get_format_instructions()
    )

    response = llm.invoke(formatted_messages)
    resume_keywords = parser.parse(response.content)


    # return 코드는 제공합니다.
    return {
        **state,
        "resume_summary": resume_summary,
        "resume_keywords": resume_keywords,
    }


# 4) 질문 전략 수립 --------------------
def generate_question_strategy(state: InterviewState) -> InterviewState:
    strategys=['경력 및 경험', '동기', '커뮤니케이션', '논리적 사고', '기술 스택', '장단점']
    quest_dirs = [
        '지원자의 전공 또는 실무 경험을 통해 직무에 필요한 역량을 쌓았는지를 평가하고, 그 경험이 직무 적합성에 어떻게 연결되는지를 확인하는 질문들.',
        '지원자가 왜 이 직무/회사에 지원했는지, 스스로의 경험이나 가치관과 어떻게 연결되는지를 파악할 수 있는 질문들.',
        '지원자가 협업 환경에서 팀에 잘 어우러질 수 있는지 파악하고, 원활한 소통 능력이 있는지 평가할 수 있는 질문들.',
        '지원자가 특정 문제를 어떻게 분석하고, 단계별로 해결 방안을 세우는지 파악할 수 있고, 특히 이력서에 기술된 프로젝트/경험을 기반으로 "왜 그렇게 판단했는지, 어떤 대안을 고려했는지" 등 논리적인 사고를 잘 하는지에 대한 질문들.',
        '지원자가 직무에 필요한 핵심 기술에 대한 기술적 능력이나 지식 수준을 보유하고 있는지 파악하고, 개념 이해, 실전 적용, 문제 해결 경험을 균형있게 갖추고 있는지 평가할 수 있는 질문들.',
        '지원자가 자기 자신에 대해 얼마나 잘 이해하고 있으며, 강점은 직무와의 연관성 중심으로, 단점은 개선 노력을 중심으로 설명할 수 있는지를 평가하는 질문들.'
    ]
    quest_examples = [
        '실무나 팀 프로젝트에서 문제 상황에 직면했을 때, 어떻게 해결했는지 구체적인 사례를 말씀해 주세요.',
        '어떤 계기로 해당 분야에 관심을 가지게 되었고, 그 관심이 지금의 지원 동기로 어떻게 연결되었는지 설명해 주세요.',
        '가장 인상깊었던 프로젝트에서 어떤 갈등이 있었고, 어떻게 극복했는지 설명해주실 수 있나요?',
        '기억에 남는 프로젝트를 진행했을 때,  문제 해결 과정에 대해 설명해 주세요. 당시 어떤 이유로 그 접근 방식을 선택했는지, 다른 해결 방안도 고려했는지 궁금합니다.',
        '이 프로젝트에서는 Python과 Pandas를 주로 사용한다고 했는데, 멀티 인덱스 처리 경험이 있나요?',
        '본인의 강점이 이 직무에서 어떻게 활용될 수 있다고 생각하시나요?'
    ] # LLM 답변 참고용

    strategy_dict = {} # 덮어쓰기 대상
    doc_lists = [] # vector db를 위한 document 리스트

    ### 6가지 전략 ###
    llm = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.6, model_kwargs={"top_p":0.8})


    for i in range(6):
      parser = CommaSeparatedListOutputParser()#

      s_msg = "너는 자소서와 이력서에 관해 질문 전략을 수집하는 유명한 면접 컨설턴트야."
      h_msg = """
      요약된 자소서/이력서 내용과 핵심 키워드를 자세히 분석하고, 질문예시를 참고해 질문전략과 질문방향에 맞는 질문을 20개 생성해줘.
      나에게 답변 시, 다음의 사항을 주의해줘.

      [답변 주의사항]
      - 부가 설명은 제외하고 질문 내용을 한문장씩 list에 담아줘.
      - 각 질문은 반드시 하나의 문장으로 끝맺고, 끝에 `?`나 "."로 끝나야 한다.
      - 한문장씩 질문내용을 list에 담을때, '.'으로 구분해야한다.

      [질문 생성 시 참고]
      - 질문전략:{strategy},
      - 질문방향:{quest_dir},
      - 요약내용:{resume_summary},
      - 핵심키워드:{resume_keywords}
      - 질문예시:{quest_example}
      """#

      chat_promt = ChatPromptTemplate.from_messages([
              ("system",s_msg), ("human",h_msg), ("system","{format_instructions}")#
      ])
      messages = chat_promt.format_messages(
          strategy=strategys[i], quest_dir=quest_dirs[i], resume_summary=state["resume_summary"],
          resume_keywords=state["resume_keywords"], quest_example=quest_examples[i],
          format_instructions=parser.get_format_instructions())#

      response = llm(messages)

      # LLM 답변 문자 필터링 및 state 저장
      tmp = parser.parse(response.content)
      quests,tmp_str = [],""
      for q in tmp:
        q=q.strip().strip(' ').strip('\n')
        if q[-1] in ['.','?']:
          if tmp_str=="":
            quests.append(" ".join(q.split(" ")[1:]))
          else:
            tmp_str+=" ".join(q.split(" ")[1:]); quests.append(tmp_str); tmp_str=""
        else:
          tmp_str+=" ".join(q.split(" ")[1:])
      strategy_dict[strategys[i]]={'질문방향':quest_dirs[i], '질문':quests, '질문회수':0}

      # 아래부터 vector_db 파일만들고, 로컬에 저장
      for q in quests:
        doc_lists.append(Document(page_content=q, metadata={'질문전략':strategys[i]}))

    embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")

    vectordb = Chroma.from_documents(
        documents=doc_lists,
        embedding=embedding_model,
        persist_directory="./chroma_db"
    )
    return {
        **state,
        "question_strategy": strategy_dict
    }




# 5) 1단계 하나로 묶기 --------------------
def preProcessing_Interview(file_path: str) -> InterviewState:
    # 파일 입력
    resume_text = extract_text_from_file(file_path)

    # state 초기화
    initial_state: InterviewState = {
        "resume_text": resume_text,
        "resume_summary": '',
        "resume_keywords": [],
        "question_strategy": {},

        "current_question": '',
        "current_answer": '',
        "current_strategy": '',
        "conversation": [],
        "evaluation": [],
        "next_step" : ''
    }

    # resume 분석
    anal_result_state = analyze_resume(initial_state)

    # 질문 전략 수립
    state = generate_question_strategy(anal_result_state)

    # 첫번째 질문 생성
    selected_question = state["question_strategy"]['경력 및 경험']['질문'][0]#

    # state 리턴
    return {
            **state,
            "current_question": selected_question,
            "current_strategy": "경력 및 경험"
            }


## ---------------- 2단계 : 면접 Agent ----------------------

# 1) 답변 입력 --------------------
def update_current_answer(state: InterviewState, user_answer: str) -> InterviewState:
    return {
        **state,
        "current_answer": user_answer.strip()
    }

# 2) 답변 평가 --------------------
def evaluate_answer(state: InterviewState) -> InterviewState:
    ## 평가 진행 ##
    evaluation = state['evaluation'] # [{'':'', ...}, {}]
    llm = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.2, model_kwargs={"top_p":0.8})

    s_msg="너는 여러 회사에서 면접 심사를 경험해본 경력 20년차 베테랑 면접관이야."
    h_msg="""
    면접자가 답변한 내용을 다음의 자료를 통해 상,중,하 중 하나로 평가를 진행해줘.
    나에게 답변 시, 답변예시처럼 답변해줘.
    - 질문내용:{current_question}
    - 답변내용:{user_answer}
    - 질문전략:{question_strategy}
    - 질문전략 부가설명:{question_strategy_direction}
    - 평가항목:질문과의 관련성, 답변의 구체성
    - 평가기준:
      - 평가항목: 질문과의 관련성
      - 해당항목 평가 기준:
        - 상 : 질문의 핵심 의도에 정확히 부합하며, 전반적인 내용을 명확히 다룸.
        - 중 : 질문과 관련은 있지만, 핵심 포인트가 부분적으로 누락됨.
        - 하 : 질문과 관련이 약하거나 엉뚱한 내용 중심.

      - 평가항목: 답변의 구체성
      - 해당항목 평가 기준:
        - 상 : 질문에 정확히 대응하며 핵심 내용이 일관성이 있음. 또한, 논리적이고 경험 중심의 구체적인 사례를 들어 설명함.
        - 중 : 경험을 언급하긴 하지만 세부 내용이 부족하거나 결과 중심이 아님.
        - 하 : 핵심 없이 두루뭉술하게 설명하거나, 질문의 요지를 제대로 파악하지 못함. 또한, 구체적인 정보가 없음.

      - 평가항목: 자기이해도
      - 해당항목 평가 기준:
        - 상 : 자신의 성격, 경험, 장단점을 명확히 이해하고 있으며, 그것이 선택에 어떻게 영향을 미쳤는지 일관되게 설명함.
        - 중 : 자기 경험과 특성을 인식하고 있으나, 깊이 있는 설명이나 구체적인 맥락이 부족함.
        - 하 : 자신의 특성이나 경험을 피상적으로 언급, 본인에 대한 이해가 부족함.

      - 평가항목: 두괄식 답변
      - 해당항목 평가 기준:
        - 상 : 첫 문장에서 요지를 명확히 전달, 이후 내용을 체계적으로 뒷반침한다. 듣는 사람이 핵심을 빠르게 이해할 수 있음.
        - 중 : 요지는 포함되어 있으나, 불분명하고 핵심이 중간이후에 드러남.
        - 하 : 핵심이 마지막에 가서야 언급이 됨. 전체적인 핵심이 무엇인지 파악하기 힘듬.

    - 답변예시
      - '저는 Python과 LangChain을 통해 AI agent를 구축해보았습니다.'
      - [중, 하, 중, 상]
      - 답변형식은 list이고 각 원소는 str형식이다.
      - 각 원소는 부가설명을 제외하고, <상/중/하> 중 하나이다.
      - list의 인덱스 0과 1번 자리는 각각 '질문과의 관련성', '답변의 구체성'을 의미한다.
      앞뒤로 ```python ~ ``` 붙이는것은 하지 마.
    """

    chat_promt = ChatPromptTemplate.from_messages([
              ("system",s_msg), ("human",h_msg),
    ])
    messages = chat_promt.format_messages(
        current_question=state["current_question"],
        user_answer=state["current_answer"],
        question_strategy=state["current_strategy"],
        question_strategy_direction=state["question_strategy"][state["current_strategy"]]['질문방향'],
    )

    response = llm(messages)
    tmp = response.content
    result = list(tmp[1:-1].split(','))

    evaluation.append({'질문과의 관련성':result[0].strip("' "), '답변의 구체성':result[1].strip("' "), '자기이해도':result[2].strip("' "), '두괄식 답변':result[3].strip("' ")})


    ## 질문 답변 내용 conversation에 추가 ##
    state["conversation"].append({'질문':state["current_question"], '답변':state["current_answer"], '질문전략':state["current_strategy"]})

    # return 코드는 제공합니다.
    return {
        **state,
        "evaluation": evaluation
    }

# 2-1) 재평가판단노드 --------------------
def reflect(state: InterviewState) -> InterviewState:
    """
    평가 결과에 대한 자기 점검 노드
    """
    last_eval = state["evaluation"][-1]
    llm = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.2)

    s_msg = "너는 스스로 평가한 면접 결과를 점검하는 자아성찰 능력을 갖춘 AI 면접관이야."
    h_msg = """
    다음은 너(면접관)가 자동으로 평가한 결과야.
    이 평가가 논리적으로 타당하고 기준에 부합하는지 점검해줘.

    - 질문: {current_question}
    - 답변: {user_answer}
    - 질문전략: {question_strategy}
    - 질문전략 부가설명: {question_strategy_direction}
    - 평가결과:
        - 질문과의 관련성: {relevance}
        - 답변의 구체성: {specificity}
        - 자기이해도 : {self_aware}
        - 두괄식 답변: {answer}

    아래 형식으로 출력하세요:
    정상
    재평가

    앞뒤로 ```python ~ ``` 붙이는것은 하지 마.
    이유는 필요없어
    """

    chat_prompt = ChatPromptTemplate.from_messages([
        ("system", s_msg),
        ("human", h_msg),
    ])

    messages = chat_prompt.format_messages(
        current_question=state["current_question"],
        user_answer=state["current_answer"],
        question_strategy=state["current_strategy"],
        question_strategy_direction=state["question_strategy"][state["current_strategy"]]["질문방향"],
        relevance=last_eval["질문과의 관련성"],
        specificity=last_eval["답변의 구체성"],
        self_aware = last_eval["자기이해도"],
        answer = last_eval["두괄식 답변"],

    )

    response = llm(messages)

    return {
        **state,
        "next_step": response.content.strip()
    }

# 2-2) 재평가노드 --------------------
def re_evaluate_answer(state: InterviewState) -> InterviewState:
    ## 평가 진행 ##
    evaluation = state['evaluation'] # [{'':'', ...}, {}]
    llm = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.2, model_kwargs={"top_p":0.8})

    s_msg="너는 여러 회사에서 면접 심사를 경험해본 경력 20년차 베테랑 면접관이야."
    h_msg="""
    면접자가 답변한 내용을 다음의 자료를 통해 상,중,하 중 하나로 평가를 진행해줘.
    나에게 답변 시, 답변예시처럼 답변해줘.
    - 질문내용:{current_question}
    - 답변내용:{user_answer}
    - 질문전략:{question_strategy}
    - 질문전략 부가설명:{question_strategy_direction}
    - 평가항목:질문과의 관련성, 답변의 구체성
    - 평가기준:
      - 평가항목: 질문과의 관련성
      - 해당항목 평가 기준:
        - 상 : 질문의 핵심 의도에 정확히 부합하며, 전반적인 내용을 명확히 다룸.
        - 중 : 질문과 관련은 있지만, 핵심 포인트가 부분적으로 누락됨.
        - 하 : 질문과 관련이 약하거나 엉뚱한 내용 중심.

      - 평가항목: 답변의 구체성
      - 해당항목 평가 기준:
        - 상 : 질문에 정확히 대응하며 핵심 내용이 일관성이 있음. 또한, 논리적이고 경험 중심의 구체적인 사례를 들어 설명함.
        - 중 : 경험을 언급하긴 하지만 세부 내용이 부족하거나 결과 중심이 아님.
        - 하 : 핵심 없이 두루뭉술하게 설명하거나, 질문의 요지를 제대로 파악하지 못함. 또한, 구체적인 정보가 없음.

      - 평가항목: 자기이해도
      - 해당항목 평가 기준:
        - 상 : 자신의 성격, 경험, 장단점을 명확히 이해하고 있으며, 그것이 선택에 어떻게 영향을 미쳤는지 일관되게 설명함.
        - 중 : 자기 경험과 특성을 인식하고 있으나, 깊이 있는 설명이나 구체적인 맥락이 부족함.
        - 하 : 자신의 특성이나 경험을 피상적으로 언급, 본인에 대한 이해가 부족함.

      - 평가항목: 두괄식 답변
      - 해당항목 평가 기준:
        - 상 : 첫 문장에서 요지를 명확히 전달, 이후 내용을 체계적으로 뒷반침한다. 듣는 사람이 핵심을 빠르게 이해할 수 있음.
        - 중 : 요지는 포함되어 있으나, 불분명하고 핵심이 중간이후에 드러남.
        - 하 : 핵심이 마지막에 가서야 언급이 됨. 전체적인 핵심이 무엇인지 파악하기 힘듬.

    - 답변예시
      - '저는 Python과 LangChain을 통해 AI agent를 구축해보았습니다.'
      - [중, 하, 중, 상]
      - 답변형식은 list이고 각 원소는 str형식이다.
      - 각 원소는 부가설명을 제외하고, <상/중/하> 중 하나이다.
      - list의 인덱스 0과 1번 자리는 각각 '질문과의 관련성', '답변의 구체성'을 의미한다.
      앞뒤로 ```python ~ ``` 붙이는것은 하지 마.
    """

    chat_promt = ChatPromptTemplate.from_messages([
              ("system",s_msg), ("human",h_msg),
    ])
    messages = chat_promt.format_messages(
        current_question=state["current_question"],
        user_answer=state["current_answer"],
        question_strategy=state["current_strategy"],
        question_strategy_direction=state["question_strategy"][state["current_strategy"]]['질문방향']
    )

    response = llm(messages)
    tmp = response.content
    result = list(tmp[1:-1].split(','))


    # 최신 평가 덮어쓰기
    evaluation[-1] ={'질문과의 관련성':result[0].strip("' "), '답변의 구체성':result[1].strip("' "), '자기이해도':result[2].strip("' "), '두괄식 답변':result[3].strip("' ")}


    # return 코드는 제공합니다.
    return {
        **state,
        "evaluation": evaluation,
        "next_step":""
    }

# 2-3) 분기 노드 --------------------
def route_reflect_next(state: InterviewState) -> Literal["re_evaluate", "decide"]:
    if state.get("next_step", "") == "재평가":
        return "re_evaluate"
    else:
        return "decide"


# 3) 인터뷰 진행 검토 --------------------
def decide_next_step(state: InterviewState) -> InterviewState:
    conversation = state.get("conversation", [])
    current_category = state.get("current_category")

    # 조건처리준비 1: 점수합계
    score_map = {"상": 3, "중": 2, "하": 1}
    total_score = sum(
        score_map.get(turn.get("질문과의 관련성"), 0) +
        score_map.get(turn.get("답변의 구체성"), 0) +
        score_map.get(turn.get("자기이해도"), 0) +
        score_map.get(turn.get("두괄식 답변"), 0)
        for turn in state.get("evaluation", [])
    )

    # 고도화 1: 모든 답변에 대해 답변의 적절성, 질문의 관련성 항목의 점수 합이 총점 96점 이상일 경우 면접을 종료한다.
    if total_score >= 96:
        next_step = "end"
    # 고도화 2: 동일 카테고리에 대한 질문이 2회 이상일 경우 다음 카테고리로 넘어간다.
    else:
        for key_ in state["question_strategy"].keys():
            if state["question_strategy"][key_].get("질문회수") < 2:
                state["current_strategy"] = key_
                break
        next_step = "additional_question"

    return {
        **state,
        "next_step": next_step
    }


# 4) 질문 생성 --------------------
def generate_question(state: InterviewState) -> InterviewState:
    # 여기에 코드를 완성합니다.
    ## 유사 문장 3개 선택 ##
    embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")
    vectordb = Chroma(
        persist_directory="./chroma_db",
        embedding_function=embedding_model
    )

    query = f"""
    metadata의 '질문전략'이 '{state['current_strategy']}'이고, 이와 유사한 질문을 알려줘.
    유사한 질문을 선택 시, 다음 문장은 제외해야 해.
    - 제외 대상 : '{state['current_question']}'

    유사 질문 선택 시, 다음의 키워드를 참고해.
    - 이력서 키워드 : {state['resume_keywords']}
    - 질문 방향 : {state['question_strategy'][state['current_strategy']]['질문방향']}
    """
    refs = vectordb.similarity_search(query, k=3)

    ## 참조 질문을 활용하여, 심화 질문 생성 ##
    llm = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.8, model_kwargs={"top_p":0.8})

    s_msg = "너는 여러 회사에서 면접 심사를 경험해본 경력 20년차 베테랑 면접관이고, 현재 면접을 진행하고 있어."
    h_msg = """
    다음의 참고 자료를 바탕으로 심화 질문 한개만 생성해줘.
    심화 질문의 목적은 지원자의 사고력, 문제해결방식, 기술적 깊이를 더 확인할 수 있는지 파악하기 위함이야.
    나에게 답변 시, 부가 설명은 제외하고 질문 내용만 말해줘.
    - 질문전략:{current_strategy},
    - 질문전략 부가설명:{question_direction},
    - 자소서/이력서 요약내용:{resume_summary},
    - 핵심키워드:{resume_keywords},
    - 이전 질문과 답변:{qna_lists},
    - 이전 답변에 대한 평가:{qna_evaluations}
    - 이전 질문과 유사한 질문 3가지:{refs}
    """

    chat_promt = ChatPromptTemplate.from_messages([
            ("system",s_msg), ("human",h_msg),
    ])
    messages = chat_promt.format_messages(
        current_strategy=state["current_strategy"], question_direction=state["question_strategy"][state["current_strategy"]]['질문방향'],
        resume_summary=state["resume_summary"], resume_keywords=state["resume_keywords"],
        qna_lists=state["conversation"][-1], qna_evaluations=state["evaluation"][-1],
        refs=refs
    )
    response = llm(messages)

    state['question_strategy'][state['current_strategy']]['질문회수']+=1

    # return 코드는 제공합니다.
    return {
        **state,
        "current_question": response.content.strip(),
        "current_answer": ""
    }




# 5) 인터뷰 피드백 보고서 --------------------
def summarize_interview(state: InterviewState) -> InterviewState:
    # 전략 분류 목록
    strategy_categories = [
        "경력 및 경험",
        "동기",
        "커뮤니케이션",
        "논리적 사고",
        "기술 스택",
        "장단점"
    ]

    categorized_data = {category: [] for category in strategy_categories}

    conversation = state["conversation"]
    evaluation = state["evaluation"]

    llm = ChatOpenAI(model_name='gpt-4o-mini', temperature=0.7)

    # (1) 질문 전략을 그대로 활용하여 분류
    for i, qa in enumerate(conversation):
        question = qa["질문"]
        answer = qa["답변"]
        strategy = qa.get("질문전략")

        categorized_data[strategy].append((question, answer, evaluation[i]))

    # (2) 전략별 피드백 생성 (요약 / 강점 / 약점)
    strategy_feedback = {}
    feedback_prompt = ChatPromptTemplate.from_messages([
        ("system", "아래 인터뷰 질문과 답변을 바탕으로 답변 요약, 강점, 약점을 작성해줘. 포맷:\n- 답변 요약:\n- 강점:\n- 약점:\n"),
        ("human", "{qna_text}")
    ])

    # strategy_feedback[category]에 결과 저장
    for category, items in categorized_data.items():
        if not items:
            continue
        qna_text = "\n".join(
            f"Q: {q}\nA: {a}" for q, a, _ in items
        )
        feedback_msg = feedback_prompt.format_messages(qna_text=qna_text)
        feedback_result = llm(feedback_msg).content.strip()
        strategy_feedback[category] = feedback_result

    # (3) 종합 피드백 생성
    overall_prompt = ChatPromptTemplate.from_messages([
        ("system", "아래 면접 기록을 바탕으로 종합 피드백을 작성해줘. 강점과 개선점을 중심으로 작성하고, 격려하는 말투로 마무리해. 모든 출력은 **와 같은 마크다운 문법 없이 일반 텍스트로 작성해줘."),
        ("human", "{full_text}")
    ])
    merged_text = "\n".join([
        f"[{category}]\n{feedback}" for category, feedback in strategy_feedback.items()
    ])
    overall_msg = overall_prompt.format_messages(full_text=merged_text)
    overall_feedback = llm(overall_msg).content.strip()

    # (4) 출력
    print("\n[인터뷰 피드백 보고서]")
    print("-" * 80)
    for category in strategy_categories:
        if category in strategy_feedback:
            print(f"\n[{category}]")
            print(strategy_feedback[category])
            print("-" * 80)

    print("\n[종합 피드백]")
    print(overall_feedback)
    print("-" * 80)

    return state


# 6) Agent --------------------
# 분기 판단 함수
def route_next(state: InterviewState) -> Literal["generate", "summarize"]:
    return "summarize" if state["next_step"] == "end" else "generate"

# 그래프 정의 시작
builder = StateGraph(InterviewState)

# 노드 추가
builder.add_node("evaluate", evaluate_answer)
builder.add_node("reflect", reflect)
builder.add_node("re_evaluate", re_evaluate_answer)
builder.add_node("decide", decide_next_step)
builder.add_node("generate", generate_question)
builder.add_node("summarize", summarize_interview)

# 노드 연결
builder.set_entry_point("evaluate")
builder.add_edge("evaluate", "reflect")
builder.add_conditional_edges("reflect", route_reflect_next)
builder.add_edge("re_evaluate", "decide")
builder.add_conditional_edges("decide", route_next)
builder.add_edge("generate", END)      # 루프
builder.add_edge("summarize", END)            # 종료

# 컴파일
graph = builder.compile()
#-------------------------------------------------------------------


########### 다음 코드는 제공되는 gradio 코드 입니다.################

import gradio as gr
import tempfile

# 세션 상태 초기화 함수
def initialize_state():
    return {
        "state": None,
        "interview_started": False,
        "interview_ended": False,
        "chat_history": []
    }

# 파일 업로드 후 인터뷰 초기화
def upload_and_initialize(file_obj, session_state):
    if file_obj is None:
        return session_state, "파일을 업로드해주세요."

    # Gradio는 file_obj.name 이 파일 경로야
    file_path = file_obj.name

    # 인터뷰 사전 처리
    state = preProcessing_Interview(file_path)
    session_state["state"] = state
    session_state["interview_started"] = True

    # 첫 질문 저장
    first_question = state["current_question"]
    session_state["chat_history"].append(["🤖 AI 면접관", first_question])

    return session_state, session_state["chat_history"]


# 답변 처리 및 다음 질문 생성
def chat_interview(user_input, session_state):
    if not session_state["interview_started"]:
        return session_state, "먼저 이력서를 업로드하고 인터뷰를 시작하세요."

    # (1) 사용자 답변 저장
    session_state["chat_history"].append(["🙋‍♂️ 지원자", user_input])
    session_state["state"] = update_current_answer(session_state["state"], user_input)

    # (2) Agent 실행 (평가 및 다음 질문 or 종료)
    session_state["state"] = graph.invoke(session_state["state"])

    # (3) 종료 여부 판단
    if session_state["state"]["next_step"] == "end":
        session_state["interview_ended"] = True
        final_summary = "✅ 인터뷰가 종료되었습니다!\n\n"
        for i, turn in enumerate(session_state["state"]["conversation"]):
            final_summary += f"\n**[질문 {i+1}]** {turn['질문']}\n**[답변 {i+1}]** {turn['답변']}\n"
            if i < len(session_state["state"]["evaluation"]):
                eval_result = session_state["state"]["evaluation"][i]
                final_summary += f"_평가 - 질문 연관성: {eval_result['질문과의 관련성']}, 답변 구체성: {eval_result['답변의 구체성']}, 자기이해도: {eval_result['자기이해도']}, 두괄식 답변: {eval_result['두괄식 답변']}_\n"
        session_state["chat_history"].append(["🤖 AI 면접관", final_summary])
        return session_state, session_state["chat_history"], gr.update(value="")

    else:
        next_question = session_state["state"]["current_question"]
        session_state["chat_history"].append(["🤖 AI 면접관", next_question])
        return session_state, session_state["chat_history"], gr.update(value="")

# Gradio 인터페이스 구성
with gr.Blocks() as demo:
    session_state = gr.State(initialize_state())

    gr.Markdown("# 🤖 AI 면접관 \n이력서를 업로드하고 인터뷰를 시작하세요!")

    with gr.Row():
        file_input = gr.File(label="이력서 업로드 (PDF 또는 DOCX)")
        upload_btn = gr.Button("인터뷰 시작")

    chatbot = gr.Chatbot()
    user_input = gr.Textbox(show_label=False, placeholder="답변을 입력하고 Enter를 누르세요.")

    upload_btn.click(upload_and_initialize, inputs=[file_input, session_state], outputs=[session_state, chatbot])
    user_input.submit(chat_interview, inputs=[user_input, session_state], outputs=[session_state, chatbot])
    user_input.submit(lambda: "", None, user_input)

# 실행
demo.launch(share=True)

Overwriting app.py


# 새 섹션

## **3. 실행**

In [ ]:
!python app.py

/content/app.py:722: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://315fee5e212d87a29e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
/content/app.py:672: UserWarning: Parameters {'top_p'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  state = preProcessing_Interview(file_path)
/content/app.py:73: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will